# Inventory Management System (Selling Portal)

**Created By :**
**Shubham Jasveer SIngh Chaudhary**

### **Import the necessary python packages**

In [ ]:
import json
import os
import time


### **Open and read the required JSON files**

In [ ]:
records_file = open('records.json', 'r+')
products = json.load(records_file)

records_file.seek(0)

# purchases.json : JSON file to store the seller transaction details
purchases_file = open('purchases.json', 'r+')

avail_qty = dict()
for prod_id, prod_info in products.items():
    avail_qty[prod_id] = prod_info['qty']
    

### **Define the necessary functions**

In [ ]:
def show_products():
    '''
    Function to display details of all products in records.json file in tabular form.
    
    Parameters:
        None
    
    Returns:
        None
    '''
    
    # Print the list of products in records.json file in tabular form
    space = " "*5
    print("\n")
    print("Product ID".center(10," ")+ space*2 + "Product Name".ljust(26," ") + space + "Price".rjust(5," ") + space + "Quantity".center(8," ") + space + "Brand Name".ljust(12," ") + space + "Category")
    for prod_id, prod_info in products.items():
        print(prod_id.center(10," ") + space + prod_info['name'].ljust(31," ") + space + str(prod_info['price']).rjust(5," ") + space + str(avail_qty[prod_id]).center(8," ") + space + prod_info['brand'].ljust(12," ") + space + prod_info['category'])

            
            
def print_product_details(id):
    '''
    Function to print details of a single product.
    
    Parameters:
        id (str) : Product ID for the product whose details is to be shown
    
    Returns:
        None
    '''
    
    print("  Product ID :", id)
    print("  Product Name :", products[id]['name'])
    print("  Price :", products[id]['price'])
    print("  Available Quantity :", avail_qty[id])
    print("  Brand Name :", products[id]['brand'])
    print("  Category :", products[id]['category'])

    
    
def add_new_product(products, transaction, sold_products):
    '''
    Function to add a new product to the product records.
    
    Parameters:
        products (json object) : JSON Object (similar to dictionary) storing all the current product details in records.json file
        
        transaction (dict)      :  Dictionary storing transaction details in the format
                                    {'trans_id' : 
                                      {'cust_name' : name, 'phone' : phone, 'address' : addr, 
                                       'prod_id' : [], 'prod_name' : [], 'qty' : [], 'price' : [],
                                       'total' : 0, 'timestamp' : ''
                                      }
                                    }
        
        sold_products (dict)   :  Dictionary storing products to be bought by customer in the format
                                    {'prod_id' : 
                                      {'prod_name' : name, 'qty' : quantity, 'price' : price}
                                    }
        
    Returns:
        None
    '''
    
    keys_list = list(map(int, list(products.keys())))   # Converting list of string to list of integers
    new_prod_id = str(min(keys_list) + len(products))
    prod_name = input("Enter Product Name : ") 
    price = int(input("Enter Price : "))
    
    qty = int(input("Enter Quantity : "))
    if(qty > 1000):      # If quantity exceeds max. storage limit
        qty = 1000
    
    brand = input("Enter Brand Name : ")
    category = input("Enter Category of Product : ")
    
    products[new_prod_id] = {'name': prod_name, 'price':price, 'qty':qty, 'brand':brand, 'category':category}
    
    sold_products[new_prod_id] = {'prod_name' : prod_name, 
                                  'qty' : qty,
                                  'price' : price
                                 }
    
    trans_id = list(transaction.keys())[0]
    transaction[trans_id]['total'] += qty*price
    avail_qty[new_prod_id] =  qty    
            

        
def search_product(attr, val):
    '''
    Function to search a product from records file using an attribute and its value.
    
    Parameters:
        attr (str) : Attribute of a product to be searched (eg.- name, brand, category)
        val  (str) : Value of the attribute to be searched
    
    Returns:
        None
    '''
    
    value = str(val).strip().lower()   # value converted to lowercase string as to perform case-insensitive search
    cnt = 0                            # Count of products satisfying provided criteria
    space = " "*5
    flag = False                       # Flag to determine if at least one product is found or not
    
    for prod_id, prod_info in products.items():
        if value in str(prod_info[attr]).strip().lower():
            cnt+=1;
            
            # Print attributes heading only once
            if(cnt == 1):
                print("\nProduct ID".center(10," ")+ space*2 + "Product Name".ljust(26," ") + space + "Price".rjust(5," ") + space + "Quantity".center(8," ") + space + "Brand Name".ljust(12," ") + space + "Category")
                flag = True
            
            print(prod_id.center(10," ") + space + prod_info['name'].ljust(31," ") + space + str(prod_info['price']).rjust(5," ") + space + str(avail_qty[prod_id]).center(8," ") + space + prod_info['brand'].ljust(12," ") + space + prod_info['category'])
            
    
    if(flag == True):
        print("\n"+str(cnt)+" product(s) found matching the criteria !!")
    else:
        print("Product not found !!")

    
    
def update_records():
    '''
    Function to update the changes in records.json file (generally after a transaction). 
    
    Parameters:
        None
    
    Returns:
        None
    '''
    
    for prod_id in products.keys():
        products[prod_id]['qty'] = avail_qty[prod_id]
    json.dump(products, records_file)
    records_file.truncate()         # Truncate to remove redundant curly braces sold in JSON file after dumping

    
    
def update_purchases(transaction, sold_products, purchases):
    '''
    Function to update the purchases.json file by appending transaction in it.
    
    Parameters:
        transaction (dict)      :  Dictionary storing transaction details in the format
                                    {'trans_id' : 
                                      {'cust_name' : name, 'phone' : phone, 'address' : addr, 
                                       'prod_id' : [], 'prod_name' : [], 'qty' : [], 'price' : [],
                                       'total' : 0, 'timestamp' : ''
                                      }
                                    }
        
        sold_products (dict)   :  Dictionary storing products to be bought by customer in the format
                                    {'prod_id' : 
                                      {'prod_name' : name, 'qty' : quantity, 'price' : price}
                                    }
        
        purchases (json object) :  JSON Object (similar to dictionary) to store transaction details in purchases.json file 
    
    Returns:
        None
    '''
    
    trans_id = list(transaction.keys())[0]
    
    for prod_id, prod_info in sold_products.items():
        transaction[trans_id]['prod_id'].append(prod_id)
        transaction[trans_id]['prod_name'].append(prod_info['prod_name'])
        transaction[trans_id]['qty'].append(prod_info['qty'])
        transaction[trans_id]['price'].append(prod_info['price'])
    
    transaction[trans_id]['timestamp'] = time.ctime()
    
    # Update purchases.json file and truncate redundant curly braces
    purchases[trans_id] = transaction[trans_id]
    json.dump(purchases, purchases_file)
    purchases_file.truncate()
    

    
def print_bill(transaction):
    '''
    Function to print the entire transaction for the seller in the form of bill.
    
    Parameters:
        transaction (dict) : Dictionary storing transaction details
    
    Returns:
        None
    '''
    
    trans_id = list(transaction.keys())[0]
    
    print("\nTransaction ID :", trans_id)
    print("Seller Name :", transaction[trans_id]['seller_name'])
    print("Phone No. :", transaction[trans_id]['phone'])
    print("Address :", transaction[trans_id]['address'])
    
    space = " "*5
    print("\nProduct ID".center(10," ")+ space + "Product Name".center(20," ") + space*3 + "Price".rjust(10," ") + space + "Quantity".center(15," ") + space + "Amount")
    for i in range(len(transaction[trans_id]['prod_id'])):
        print(transaction[trans_id]['prod_id'][i].center(10," ") + space + 
              transaction[trans_id]['prod_name'][i].ljust(35," ") + space + 
              str(transaction[trans_id]['price'][i]).rjust(5," ") + space + 
              str(transaction[trans_id]['qty'][i]).center(15," ") + space + 
              str(transaction[trans_id]['price'][i] * transaction[trans_id]['qty'][i]))
    
    print("\nTotal Amount :",transaction[trans_id]['total'])

## **Main Program**

In [ ]:
print("Please fill in the details before selling products.")
name = input("Seller Name : ")
phone = input("Phone No. : ")
addr = input("Address : ")


# Check if purchases.json file is empty as json.load() function fails for empty file
# For empty file, we create an empty 'purchases' dictionary to put in purchases.json file
purchases = {}
if (os.stat('purchases.json').st_size == 0):   
    trans_id = '1'
else:
    purchases = json.load(purchases_file)
    purchases_file.seek(0)
    trans_id = str(len(purchases) + 1)
    
# Dictionary to store transaction details and to be appended to purchases.json file at end
transaction = {trans_id : {'seller_name' : name, 'phone' : phone, 'address' : addr, 
                           'prod_id' : [], 
                           'prod_name' : [],
                           'qty' : [],
                           'price' : [],
                           'total' : 0,
                           'timestamp' : ''
                          }
              }


# Dictionary to store details of each sold product as {prod_id : {prod_name : ____, qty : ____, price : ____}} 
sold_products = {}


while(True):
    # Menu for features provided to the customer
    print("\n")
    print(" MENU ".center(30,'*'))
    print("1. Display list of products")
    print("2. Search for a product")
    print("3. Sell a product (already present in records)")
    print("4. Sell a new product")
    print("5. Finish the transaction")
    print("6. Clear the transaction")
    print("Press 0 to exit")
    choice = int(input("\nEnter your choice (0-6) : "))
    
    if (choice == 0):
        print("\nNo transaction made!!")
        print("Exited successfully !!")
    
    
    elif (choice == 1):
        show_products()
    
    
    elif (choice == 2):
        # Menu to ask user to select a criteria to search for a product
        print("On what basis would you like to search the product?")
        print("1. Product ID")
        print("2. Product Name")
        print("3. Brand")
        print("4. Category")
        option = int(input("\nEnter your choice (1-4) : "))
        
        if (option == 1):
            id = input("\nEnter Product ID : ")
            if id in products:
                print("Product Found !!")
                print_product_details(id)
            else:
                print("Product not found !!")
        
        elif (option == 2):
            name = input("Enter Product Name : ")
            search_product('name', name)
        
        elif (option == 3):
            brand = input("Enter Brand Name : ")
            search_product('brand', brand)
        
        elif (option == 4):
            category = input("Enter Category : ")
            search_product('category', category)
        
        else:
            print("Wrong Choice !!")
    
    
    elif (choice == 3):
        while(True):
            prod_id = input("\nEnter Product ID of the product to sell : ")
            
            if prod_id in products:
                if avail_qty[prod_id] == 1000:   # Max. quantity limit for storage of a particular product 
                    print("Sorry!! Stock is already full.")
                
                else:
                    print_product_details(prod_id) 
                    qty = int(input("\nEnter the quantity to sell : "))                    
                    
                    if (qty <= 0):
                        print("Invalid quantity !!")
                        print("Please try again with valid quantity.")
                    
                    elif (qty + avail_qty[prod_id] > 1000):
                        print("Quantity to sell exceeds the available storage capacity !!")
                        print("Please try again with lower quantity.")
                    
                    else:
                        print("Product added successfully !!")
                        print("Amount :", qty*products[prod_id]['price'])
                        avail_qty[prod_id] += qty
                        
                        # Check if product is already in sold products (ie. sold earlier)
                        if prod_id in sold_products:
                            sold_products[prod_id]['qty'] += qty
                        else:
                            sold_products[prod_id] = {'prod_name' : products[prod_id]['name'], 
                                                      'qty' : qty,
                                                      'price' : products[prod_id]['price']
                                                     }
          
                        transaction[trans_id]['total'] += qty*products[prod_id]['price']
            
            else:
                print("Invalid Product ID !!")
            
            ch = input("\nWould you like to sell more products? (y/n) : ")
            # If any character other than 'y' or 'Y', then no further purchasing to be done.
            if(ch=='y' or ch=='Y'):        
                continue
            else:
                break
    
    
    elif (choice == 4):
        add_new_product(products, transaction, sold_products)
        print("\nNew product added successfully !!")
        
        if(transaction[trans_id]['total'] != 0):
            update_records()
            update_purchases(transaction, sold_products, purchases)
            print_bill(transaction)
            print("\nThank you and hope to see you again!!")
            
        else:
            print("\nTransaction finished. No products added !!")
        
        break
    
    
    elif (choice == 5):
        # Check if seller has sold something. If yes, then only update the records.json and purchases.json files
        if(transaction[trans_id]['total'] != 0):
            print("\nTransaction finished successfully!!")
            update_records()
            update_purchases(transaction, sold_products, purchases)
            print_bill(transaction)
            print("\nThank you and hope to see you again!!")
            
        else:
            print("\nTransaction finished. No products sold !!")
        
        break
    
    
    elif (choice == 6):
        # To cancel transaction means reinitialize available qty. with original qty. from products dictionary  
        for prod_id in products.keys():
            avail_qty[prod_id] = products[prod_id]['qty']
        
        # Clear the sold_products dictionary and reset the transaction's total cost to 0
        sold_products.clear()
        transaction[trans_id]['total'] = 0
        print("\nTransaction cleared successfully !!")
    
    
    else:
        print("\nWrong Choice !!")

     
# Close all opened files before exiting program
records_file.close()
purchases_file.close()
    